In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
import gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with open('/kaggle/input/flicktime/movielens1.json', 'r') as f:
    data = json.load(f)

tmdb_data = pd.json_normalize(data)

In [3]:
link = pd.read_csv('/kaggle/input/flicktime/link.csv')
rating = pd.read_csv('/kaggle/input/flicktime/rating.csv')

In [4]:
tmdb_data.drop(['belongs_to_collection', 'belongs_to_collection.backdrop_path', 'belongs_to_collection.poster_path',
               'belongs_to_collection.name','belongs_to_collection.id'],inplace = True, axis = 1)

tmdb_data.dropna(inplace = True)

In [5]:
merged_df = tmdb_data.merge(link,left_on='id', right_on = 'tmdbId')

In [6]:
del tmdb_data
gc.collect()

0

In [7]:
merged_df.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId,tmdbId
0,False,/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Toy Story,False,7.969,16648,1,114709,862.0
1,False,/pYw10zrqfkdm3yD9JTO6vEGQhKy.jpg,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.sonypictures.com/movies/jumanji/,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Roll the dice and unleash the excitement!,Jumanji,False,7.238,9580,2,113497,8844.0
2,False,/1J4Z7VhdAgtdd97nCxY7dcBpjGT.jpg,25000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.458,325,3,113228,15602.0
3,False,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.273,132,4,114885,31357.0
4,False,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,6.243,640,5,113041,11862.0


In [8]:
merged_df[merged_df['title'].str.startswith('Thor')]


,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId,tmdbId
5899,False,/ufekY2p0zR41eAeKDrqpmylDa6K.jpg,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",,32489,tt0062362,en,Thoroughly Modern Millie,"Millie Dillmount, a fearless young lady fresh ...",...,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,Julie as you love her... in the happiest motio...,Thoroughly Modern Millie,False,6.900,74,6480,62362,32489.0
15543,False,/1wOu8rdvPxU1ObHi20VcRhfNpbo.jpg,150000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://thor.marvel.com/,10195,tt0800369,en,Thor,"Against his father Odin's will, The Mighty Tho...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Two worlds. One hero.,Thor,False,6.767,19559,86332,800369,10195.0
18620,False,/sYtby8VBJnEBG6X9dH3CVIgzQO2.jpg,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",,63686,tt1667903,en,Thor: Tales of Asgard,"Hungry for adventure, Thor secretly embarks on...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Before the hammer ... came the sword,Thor: Tales of Asgard,False,6.368,148,102009,1667903,63686.0
19484,False,/4zTsF0RtO0av3YX1NaKDqGKPxYF.jpg,170000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.marvel.com/movies/thor-the-dark-world,76338,tt1981115,en,Thor: The Dark World,Thor fights to restore order across the cosmos...,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Delve into the darkness,Thor: The Dark World,False,6.546,15962,106072,1981115,76338.0
22618,False,/uQdkjMxNxEUfegkoZ9pe9f0G1k3.jpg,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,78354,tt0029664,en,Thoroughbreds Don't Cry,Cricket West is a hopeful actress with a plan ...,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Racing right into your heart,Thoroughbreds Don't Cry,False,6.200,5,124013,29664,78354.0


In [9]:
merged_rating = rating.loc[rating['movieId'].isin(merged_df.movieId.values)]

In [10]:
merged_rating.userId.max()

138493

In [11]:
merged_rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [12]:
# try:
#     value = merged_rating.loc[(merged_rating['userId'] == 1) & (merged_rating['movieId'] == 100000000), 'rating'].iloc[0]
# except IndexError:
#     value = None

In [13]:
# print(value)

In [14]:
# user_df = pd.DataFrame(columns=['email','userId'])

In [15]:
# max( merged_rating[['userID']])

In [16]:
# pickle.dump(user_df, open('user_df.pkl', 'wb'))

In [17]:
# user_df.columns

In [18]:
# user_df.loc[user_df['email'] == email, 'userId'].iloc[0] if not user_df.loc[user_df['email'] == email].empty else None

In [19]:
# userid = df.loc[df['Email'] == email, 'UserID'].iloc[0] if not df.loc[df['Email'] == email].empty else None


In [20]:
# merged_rating.head()

In [21]:
# # Original dictionary with string keys
# original_dict = {'1': 3.5, '2': 4.5, '3': 5}

# # Create a new dictionary with integer keys
# new_dict = {}
# for key, value in original_dict.items():
#     new_dict[int(key)] = value

# # Print the new dictionary
# print(new_dict)


In [22]:
# def add_rating(merged_rating, ratingsMap, userId):
#     # Create a new dictionary with integer keys
#     new_dict = {}
#     for key, value in ratingsMap.items():
#         new_dict[int(key)] = value
        
#     # Filter the merged_rating dataframe for the specific user and the movies in the rating map
#     user_ratings = merged_rating[(merged_rating['userId'] == userId) & (merged_rating['movieId'].isin(ratingsMap.keys()))]
#     print(user_ratings)
    
#     # Loop over the movie IDs in the rating map
#     for movie_id in ratingsMap.keys():
#         # Check if the movie ID is already in the user_ratings dataframe
#         if movie_id not in user_ratings['movieId'].values:
#             # If not, add the new rating as a new row to the user_ratings dataframe
#             new_row = pd.DataFrame({'userId': [userId], 'movieId': [movie_id], 'rating': [ratingsMap[movie_id]], 'timestamp': [pd.Timestamp.now()]})
#             user_ratings = user_ratings.append(new_row)
            
# #     print(user_ratings)
#     # Update the merged_rating dataframe with the updated user_ratings dataframe
#     merged_rating.update(user_ratings)
#     print(merged_rating.head())

In [23]:
# add_rating(merged_rating, original_dict, 1)

In [24]:
# Define a TF-IDF vectorizer for the overview field
tfidf = TfidfVectorizer(stop_words='english')

# Compute the TF-IDF matrix for the overviews
tfidf_matrix = tfidf.fit_transform(merged_df['overview'].fillna(''))

In [25]:
# Compute the cosine similarities between movies based on the overviews
cosine_similarities = cosine_similarity(tfidf_matrix)

In [26]:
del tfidf_matrix

gc.collect()

0

In [27]:
# pickle.dump(cosine_similarities, open('cosine_similarities.pkl', 'wb'))

In [28]:
# pickle.dump(merged_rating, open('merged_rating.pkl', 'wb'))

In [29]:
# pickle.dump(merged_df, open('merged_df.pkl', 'wb'))

In [30]:
# temp = pickle.load(open('/kaggle/working/merged_rating.pkl', 'rb'))

In [31]:
# temp1 = pickle.load(open('/kaggle/working/merged_df.pkl', 'rb'))

In [32]:
# del temp, temp1
# gc.collect()

In [33]:
# Define the number of recommendations to make
num_recommendations = 20

In [34]:
print(1 in merged_rating['userId'].values)

True


In [35]:
def get_hybrid_recommendations(user_id, watch_history, collaborative_model, content_model):
    # Get the user's movie history
    user_history = merged_rating[merged_rating['userId'] == user_id]['movieId'].unique()
    
    temp = []
    for i in watch_history:
        temp.extend(merged_df.loc[merged_df['id'] == i]['movieId'].values)
    
    watch_history = np.array(temp)
    
    if len(watch_history) > 0:
        print('append')
        user_history = np.append(user_history, watch_history)
    
    user_history = np.unique(user_history)
    
    # Make predictions for all unseen movies using the collaborative model
    unseen_movies = np.setdiff1d(merged_df['movieId'].unique(), user_history)
    
    test_movie_ids = np.array(unseen_movies)
    test_user_ids = np.array(len(unseen_movies)*[user_id])
    
    test_input = [test_user_ids, test_movie_ids]
    unseen_ratings = model_nn_84.predict(test_input).flatten()
    
    unseen_indices = np.argsort(unseen_ratings)[::-1][:num_recommendations]
    collaborative_recommendations = unseen_movies[unseen_indices]
    
    # Get the top similar movies to the user's history using the content model
    content_recommendations = []
    for movie_id in user_history:
        movie_index = merged_df[merged_df['movieId'] == movie_id].index[0]
        similar_indices = cosine_similarities[movie_index].argsort()[::-1][1:num_recommendations+1]
        content_recommendations += list(merged_df.iloc[similar_indices]['movieId'].values)
    content_recommendations = np.array(content_recommendations)
    
    
    
    # Combine the two lists of recommendations
    hybrid_recommendations = np.array(np.union1d(collaborative_recommendations, content_recommendations))

    hybrid_test_user_ids = np.array(len(hybrid_recommendations)*[user_id])
    
    hybrid_test_input = [hybrid_test_user_ids, hybrid_recommendations]

    hybrid_ratings = model_nn_84.predict(hybrid_test_input).flatten()
    
    hybrid_indices = np.argsort(hybrid_ratings)[::-1][:num_recommendations]
    hybrid_recommendations = hybrid_recommendations[hybrid_indices]
    
    del user_history, unseen_movies, unseen_ratings, unseen_indices, collaborative_recommendations, content_recommendations, hybrid_ratings, hybrid_indices
    gc.collect()
    
    return hybrid_recommendations



In [36]:
model_nn_84 = pickle.load(open('/kaggle/input/flicktime/model_nn_84.pkl', 'rb'))

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-05-10 10:56:44         4604
variables.h5                                   2023-05-10 10:56:44     32496008
metadata.json                                  2023-05-10 10:56:44           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......concatenate
.........vars
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......dropout_2
.........vars
......embedding
.........vars
............0
......embedding_1
.........vars
............0
......flatten
.........vars
......flatten_1
.........vars
......input_layer
.........vars
......input_layer_1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimize

In [37]:
user_id = 1
watch_history = [434, 85, 12506, 16320, 186, 388]
hybrid_recommendations = get_hybrid_recommendations(user_id, watch_history, model_nn_84, cosine_similarities)

append
95/95 [==============================] - 0s 2ms/step


In [39]:
recommended_movies = merged_df[merged_df['movieId'].isin(hybrid_recommendations)]
recommended_movies[['id','overview','title']]

# 434, 85, 12506, 16320, 186, 388

,id,overview,title
494,424,The true story of how businessman Oskar Schind...,Schindler's List
1064,85,When Dr. Indiana Jones – the tweed-suited prof...,Raiders of the Lost Ark
2048,73,Derek Vineyard is paroled after serving 3 year...,American History X
2274,603,"Set in the 22nd century, The Matrix tells the ...",The Matrix
2918,8374,Tired of the crime overrunning the streets of ...,The Boondock Saints
6520,122,Aragorn is revealed as the heir to the ancient...,The Lord of the Rings: The Return of the King
8298,17663,At the turn of the century on Prince Edward Is...,Anne of Green Gables
9210,42994,"In this anime anthology, a salvage ship crew h...",Memories
11040,5723,A vacuum repairman moonlights as a street musi...,Once
11629,155,Batman raises the stakes in his war on crime. ...,The Dark Knight
